In [1]:
# 필요한 라이브러리 불러오기
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

In [2]:
# 모델 및 데이터 경로 설정
model1_path = "방법 3/model/model1_E_vs_NotE.cbm"
model2_path = "방법 3/model/model2_3class.cbm"
model3_path = "방법 3/model/model3_A_vs_B.cbm"
data_path = "피처_43/피처 선택/43_Segment 추가.csv"

In [3]:
# 모델 불러오기
model1 = CatBoostClassifier(); model1.load_model(model1_path)
model2 = CatBoostClassifier(); model2.load_model(model2_path)
model3 = CatBoostClassifier(); model3.load_model(model3_path)

In [4]:
# 데이터 불러오기 및 feature 컬럼 정의
df_full = pd.read_csv(data_path)
feature_cols = df_full.columns.drop(['ID', 'Segment'])

In [5]:
# 🔹 model1: E vs NotE 예측
X_full = df_full[feature_cols]
pred1 = model1.predict(X_full)
df_full['Segment_pred'] = pred1

import pickle

# 숫자 → 문자 매핑
label_map_model1 = {1: 'E', 0: 'NotE'}

# 매핑 적용
df_full['Segment_pred'] = pd.Series(pred1.flatten()).map(label_map_model1)


df_e = df_full[df_full['Segment_pred'] == 'E']
df_not_e = df_full[df_full['Segment_pred'] != 'E']
X_not_e = df_not_e[feature_cols]

In [6]:
pred2 = model2.predict(df_not_e[feature_cols])
df_stage2 = df_not_e.copy()
df_stage2['Segment_pred'] = pred2.ravel()
df_cd = df_stage2[df_stage2['Segment_pred'].isin(['C', 'D'])].copy()

In [7]:
# 🔹 model3 예측 준비 + 오류 완전 방지
df_not_cd = df_stage2[~df_stage2['Segment_pred'].isin(['C', 'D'])].copy()
df_not_cd.reset_index(drop=True, inplace=True)

if not df_not_cd.empty:
    X_not_cd = df_not_cd[feature_cols]
    pred3 = model3.predict(X_not_cd)
    df_not_cd['Segment_pred'] = pred3
else:
    df_not_cd['Segment_pred'] = pd.Series([], dtype=object)


In [8]:
# 🔹 model3: A vs B 예측
pred3 = model3.predict(X_not_cd)
df_not_cd = df_not_cd.copy()
df_not_cd['Segment_pred'] = pred3

In [9]:
# 🔹 최종 예측 결과 병합
df_final = pd.concat([df_e, df_cd, df_not_cd], ignore_index=True)

In [10]:
# 🔹 NaN 제거 및 dtype 통일 후 F1-micro 계산
df_eval = df_final.dropna(subset=['Segment', 'Segment_pred']).copy()
df_eval['Segment'] = df_eval['Segment'].astype(str)
df_eval['Segment_pred'] = df_eval['Segment_pred'].astype(str)

from sklearn.metrics import f1_score
f1_micro = f1_score(df_eval['Segment'], df_eval['Segment_pred'], average='micro')
print(f"✅ 전체 다단계 모델 F1_micro: {f1_micro:.4f}")

✅ 전체 다단계 모델 F1_micro: 0.9474
